# Using Reddit's API for Predicting Comments

In this project, we will practice two major skills. Collecting data via an API request and then building a binary predictor.

As we discussed in week 2, and earlier today, there are two components to starting a data science problem: the problem statement, and acquiring the data.

For this article, your problem statement will be: _What characteristics of a post on Reddit contribute most to the overall interaction (as measured by number of comments)?_

Your method for acquiring the data will be scraping the 'hot' threads as listed on the [Reddit homepage](https://www.reddit.com/). You'll acquire _AT LEAST FOUR_ pieces of information about each thread:
1. The title of the thread
2. The subreddit that the thread corresponds to
3. The length of time it has been up on Reddit
4. The number of comments on the thread

Once you've got the data, you will build a classification model that, using Natural Language Processing and any other relevant features, predicts whether or not a given Reddit post will have above or below the _median_ number of comments.

**BONUS PROBLEMS**
1. If creating a logistic regression, GridSearch Ridge and Lasso for this model and report the best hyperparameter values.
1. Scrape the actual text of the threads using Selenium (you'll learn about this in Webscraping II).
2. Write the actual article that you're pitching and turn it into a blog post that you host on your personal website.

### Scraping Thread Info from Reddit.com

#### Set up a request (using requests) to the URL below. 

*NOTE*: Reddit will throw a [429 error](https://httpstatuses.com/429) when using the following code:
```python
res = requests.get(URL)
```

This is because Reddit has throttled python's default user agent. You'll need to set a custom `User-agent` to get your request to work.
```python
res = requests.get(URL, headers={'User-agent': 'YOUR NAME Bot 0.1'})
```

In [20]:
import requests
import json
import numpy as np 
from bs4 import BeautifulSoup
import pandas as pd
import re
from  time import sleep

In [ ]:
# URL = "http://www.reddit.com/hot.json"

In [34]:
## YOUR CODE HERE
df = pd.DataFrame(columns=['Title', 'Subreddit', 'Number of Comments', 'Time', 'Domain', 'Score'])
end_page=''
for x in range(50):
    try:
        url = "https://www.reddit.com/r/popular/hot/"+end_page
        response = requests.get(url, headers={'User-Agent': 'DontWorry'})
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        titles = [title.text for title in soup.find_all('a', {'data-event-action': 'title'})]
        subreddits = [sub.text for sub in soup.find_all('a', {'class': 'subreddit hover may-blank'})]
        comments = [comment.text for comment in soup.find_all('a', {'class': 'bylink comments may-blank'})]
        time = [time['title'] for time in soup.find_all('time')]  
        votes = [score.text for score in soup.find_all('div', {'class': 'score likes'})]
        end_tags = soup.find_all('div', {'class':'reportform'})
        last_tag=end_tags[24]['class'][1].split('-')[1]
        end_page="?count=50&after="+last_tag                                
        df = pd.DataFrame({'Title': titles, 'Subreddit': subreddits, 'Number of Comments': comments, 'Time': time, 'Votes': votes})
        reddit = reddit.append(df)
        df.to_csv('reddit.csv')
    except:
        pass

#### Use `res.json()` to convert the response into a dictionary format and set this to a variable. 

```python
data = res.json()
```

In [30]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   popular links
  </title>
  <meta content=" reddit, reddit.com, vote, comment, submit " name="keywords"/>
  <meta content="Reddit gives you the best of the internet in one place. Get a constantly updating feed of breaking news, fun stories, pics, memes, and videos just for you. Passionate about something niche? Reddit has thousands of vibrant communities with people that share your interests. Alternatively, find out what’s trending across all of Reddit on r/popular. Reddit is also anonymous so you can be yourself, with your Reddit profile and persona disconnected from your real-world identity." name="description"/>
  <meta content="always" name="referrer"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="/static/opensearch.xml" rel="search" type="application/opensearchdescription+xml"/>
  <link href="https://www.reddit.com/r/popular/" rel="canonic

In [14]:
end_tags

[<div class="reportform report-t3_8lzvm0"></div>,
 <div class="reportform report-t3_8m0w1z"></div>,
 <div class="reportform report-t3_8lya81"></div>,
 <div class="reportform report-t3_8lxboj"></div>,
 <div class="reportform report-t3_8lwlbl"></div>,
 <div class="reportform report-t3_8lxx6i"></div>,
 <div class="reportform report-t3_8m0vkl"></div>,
 <div class="reportform report-t3_8ly35r"></div>,
 <div class="reportform report-t3_8lxg52"></div>,
 <div class="reportform report-t3_8lwoow"></div>,
 <div class="reportform report-t3_8lvwp1"></div>,
 <div class="reportform report-t3_8m08tu"></div>,
 <div class="reportform report-t3_8m0uwm"></div>,
 <div class="reportform report-t3_8lxf7r"></div>,
 <div class="reportform report-t3_8lx5e2"></div>,
 <div class="reportform report-t3_8lztw2"></div>,
 <div class="reportform report-t3_8m0849"></div>,
 <div class="reportform report-t3_8m083w"></div>,
 <div class="reportform report-t3_8ly8c8"></div>,
 <div class="reportform report-t3_8luw23"></div>,


In [15]:
len(end_tags)

25

In [16]:
last_tag

't3_8lx7nx'

In [17]:
 end_tags1 = soup.find_all('div', {'class':'reportform'})[-1][1].split('-')[-1]
#         last_tag1=end_tags[24]['class'][1].split('-')[1]

KeyError: 1

In [32]:
df.shape

(25, 5)

In [33]:

reddit = pd.read_csv('reddit.csv')

FileNotFoundError: File b'reddit.csv' does not exist

In [8]:
reddit.head()

,Unnamed: 0,Domain,Number of Comments,Score,Scores,Subreddit,Time,Title
0,0,(bbc.co.uk),1184 comments,26.5k,NaN,r/worldnews,Fri May 25 08:38:33 2018 UTC,Australia and Netherlands formally accuse Russ...
1,1,(i.redd.it),150 comments,9105,NaN,r/pics,Fri May 25 10:26:05 2018 UTC,'Raising Men Lawn Care' is a group of young me...
2,2,(i.redd.it),198 comments,11.1k,NaN,r/OldSchoolCool,Fri May 25 08:21:53 2018 UTC,"My favourite picture of my Dad, when living in..."
3,3,(i.redd.it),96 comments,12.4k,NaN,r/aww,Fri May 25 07:44:55 2018 UTC,spot the odd one
4,4,(i.imgur.com),16 comments,2816,NaN,r/AnimalTextGifs,Fri May 25 10:00:42 2018 UTC,"""There you go little human, you got it."""


#### Getting more results

By default, Reddit will give you the top 25 posts:

```python
print(len(data['data']['children']))
```

If you want more, you'll need to do two things:
1. Get the name of the last post: `data['data']['after']`
2. Use that name to hit the following url: `http://www.reddit.com/hot.json?after=THE_AFTER_FROM_STEP_1`
3. Create a loop to repeat steps 1 and 2 until you have a sufficient number of posts. 

*NOTE*: Reddit will limit the number of requests per second you're allowed to make. When you create your loop, be sure to add the following after each iteration.

```python
time.sleep(3) # sleeps 3 seconds before continuing```

This will throttle your loop and keep you within Reddit's guidelines. You'll need to import the `time` library for this to work!

In [ ]:
## YOUR CODE HERE

## (Optional) Collect more information

While we only require you to collect four features, there may be other info that you can find on the results page that might be useful. Feel free to write more functions so that you have more interesting and useful data.

In [ ]:
## YOUR CODE HERE

### Save your results as a CSV
You may do this regularly while scraping data as well, so that if your scraper stops of your computer crashes, you don't lose all your data.

In [ ]:
# Export to csv


## Predicting comments using Random Forests + Another Classifier

#### Load in the the data of scraped results

In [ ]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the number of comments was low or high. Compute the median number of comments and create a new binary variable that is true when the number of comments is high (above the median)

We could also perform Linear Regression (or any regression) to predict the number of comments here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW number of comments.

While performing regression may be better, performing classification may help remove some of the noise of the extremely popular threads. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of comment numbers. 

In [ ]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [ ]:
## YOUR CODE HERE

#### Create a Random Forest model to predict High/Low number of comments using Sklearn. Start by ONLY using the subreddit as a feature. 

In [ ]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a thread title.
- For example, create a feature that represents whether 'cat' is in the title or whether 'funny' is in the title. 
- Then build a new Random Forest with these features. Do they add any value?
- After creating these variables, use count-vectorizer to create features based on the words in the thread titles.
- Build a new random forest model with subreddit and these new features included.

In [ ]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy of the model, as well as any other metrics you feel are appropriate. 

In [ ]:
## YOUR CODE HERE

#### Repeat the model-building process with a non-tree-based method.

In [ ]:
## YOUR CODE HERE

#### Use Count Vectorizer from scikit-learn to create features from the thread titles. 
- Examine using count or binary features in the model
- Re-evaluate your models using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

# Executive Summary
---
Put your executive summary in a Markdown cell below.

### BONUS
Refer to the README for the bonus parts

In [ ]:
## YOUR CODE HERE